In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
import shapely as sp

First, the corrdidor needs to be defined

In [ ]:
study_corridor = sp.LineString([
[36.172843, -86.765355],
[36.177680, -86.752774],
[36.178151, -86.751648],
[36.178650, -86.750863],
[36.179258, -86.750354],
[36.179961, -86.749966],
[36.191913, -86.744805],
[36.203607, -86.738496],
[36.205022, -86.737587],
[36.206045, -86.736810],
[36.210328, -86.733471],
[36.215705, -86.729604],
[36.216632, -86.729000],
[36.222668, -86.725808],
[36.232123, -86.724801],
[36.234704, -86.724745]
])
corridor = sp.to_geojson(study_corridor)
# So far only mapped from Spring St to Briley

In [ ]:
zipcodes = gpd.read_file('../data/clean/zipcodes.geojson')
zipcodes = zipcodes[zipcodes['zip'].isin(['37206', '37216', '37115'])]
zipcodes
center = zipcodes.geometry.centroid[1]
map_center = [center.y, center.x]

In [ ]:
all_crashes_no_hwys = pd.read_csv('../data/clean/all_crashes_no_hwys.csv')
zipcodes = gpd.read_file('../data/clean/zipcodes.geojson')
county = gpd.read_file('../data/clean/Davidson County Border (GIS).geojson')
county

Create a heat map of all the crashes 

In [ ]:
all_crashes_no_hwys = folium.Map(location = map_center, tiles="Cartodb Positron", zoom_start = 12)

folium.GeoJson(zipcodes, style_function=lambda feature: {"color": "black", "weight": 2, "dashArray": "10, 5","fillOpacity":0.125}).add_to(all_crashes_no_hwys)

crashes = []
for row_index, row_values in crashes_no_hwys_geo.iterrows():
    loc = [row_values['lat'], row_values['long']]
    crashes.append(loc)
crashes

HeatMap(crashes, radius = 15, min_opacity = 0.5, gradient={.5: '#ffc2c2', .75: '#ff7970', 1: '#ff0000'}).add_to(folium.FeatureGroup(name='Heat Map').add_to(all_crashes_no_hwys))
folium.LayerControl().add_to(all_crashes_no_hwys)

all_crashes_no_hwys.save('../maps/all_crashes_no_hwys.html')

Updating the map to differentiate collision types.

Now let's see where there are 311 requests along the corridor.

In [ ]:
nash_311 = pd.read_csv('../data/clean/nash_311.csv')

In [ ]:
nash_311_geo = nash_311[['case_number', 'latitude', 'longitude']]
nash_311_geo = gpd.GeoDataFrame(nash_311_geo, crs= zipcodes.crs, geometry=gpd.points_from_xy(nash_311_geo.longitude, nash_311_geo.latitude))

In [ ]:
nash_311 = folium.Map(location = map_center, tiles="Cartodb Positron", zoom_start = 12)

folium.GeoJson(zipcodes, style_function=lambda feature: {"color": "black", "weight": 2, "dashArray": "10, 5","fillOpacity":0.125}).add_to(nash_311)

requests_311 = []
for row_index, row_values in nash_311_geo.iterrows():
    loc = [row_values['latitude'], row_values['longitude']]
    requests_311.append(loc)
requests_311

HeatMap(requests_311, radius = 15, min_opacity = 0.5, gradient={.5: '#98ABEE', .75: '#1D24CA', 1: '#201658'}).add_to(folium.FeatureGroup(name='Heat Map').add_to(nash_311))
folium.LayerControl().add_to(nash_311)

nash_311.save('../maps/nash_311.html')